In [51]:
import os

import warnings as warnings
warnings.filterwarnings(action='ignore')

import numpy as np

import pandas as pd

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', -1)
pd.options.display.float_format = '{:.2f}'.format # 소수점 유효숫자 표기법으로 안쓰기

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

import lightgbm as lgb
from bayes_opt import BayesianOptimization

import matplotlib.pyplot as plt

# from tqdm import tqdm                       # 진행바
from sklearn.model_selection import KFold   # K-fold CV
from functools import partial

In [52]:
# data load
data_folder = 'model_data/'

train_final_ftr_df = pd.read_csv(os.path.join(data_folder,'train_final_ftr_0414_5.csv'))
# test_final_ftr_df = pd.read_csv(os.path.join(data_folder,'test_final_ftr_0410.csv'))

train_final_ftr_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_final_ftr_df.columns]
# test_final_ftr_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_final_ftr_df.columns]

# train_final_ftr_df['map_0'] = train_final_ftr_df['map_0'].astype('int').astype('category')
# train_final_ftr_df['map_1'] = train_final_ftr_df['map_1'].astype('int').astype('category')
# train_final_ftr_df['species_0'] = train_final_ftr_df['species_0'].astype('int').astype('category')
# train_final_ftr_df['species_1'] = train_final_ftr_df['species_1'].astype('int').astype('category')

# test_final_ftr_df['map_0'] = test_final_ftr_df['map_0'].astype('int').astype('category')
# test_final_ftr_df['map_1'] = test_final_ftr_df['map_1'].astype('int').astype('category')
# test_final_ftr_df['species_0'] = test_final_ftr_df['species_0'].astype('int').astype('category')
# test_final_ftr_df['species_1'] = test_final_ftr_df['species_1'].astype('int').astype('category')

In [15]:
# data load
data_folder = 'model_data/'

train_final_ftr_df = pd.read_csv(os.path.join(data_folder,'train_final_ftr_0414_4_inner.csv'))
# test_final_ftr_df = pd.read_csv(os.path.join(data_folder,'test_final_ftr_0410.csv'))

train_final_ftr_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_final_ftr_df.columns]
# test_final_ftr_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_final_ftr_df.columns]

# train_final_ftr_df['map_0'] = train_final_ftr_df['map_0'].astype('int').astype('category')
# train_final_ftr_df['map_1'] = train_final_ftr_df['map_1'].astype('int').astype('category')
# train_final_ftr_df['species_0'] = train_final_ftr_df['species_0'].astype('int').astype('category')
# train_final_ftr_df['species_1'] = train_final_ftr_df['species_1'].astype('int').astype('category')

# test_final_ftr_df['map_0'] = test_final_ftr_df['map_0'].astype('int').astype('category')
# test_final_ftr_df['map_1'] = test_final_ftr_df['map_1'].astype('int').astype('category')
# test_final_ftr_df['species_0'] = test_final_ftr_df['species_0'].astype('int').astype('category')
# test_final_ftr_df['species_1'] = test_final_ftr_df['species_1'].astype('int').astype('category')

In [54]:
## 전체 데이터 가져오기
_ftr = train_final_ftr_df.columns.tolist()
_ftr.remove('game_id')
_ftr.remove('winner')

# train_data, test_data = train_test_split(train_final_ftr_df, test_size=0.2, random_state=1)

# X_train, y_train = train_data[_ftr], train_data['winner']
# X_test, y_test = test_data[_ftr], test_data['winner']

In [64]:
df = pd.read_csv('model_data/train_final_ftr_0414_5.csv')

In [65]:
df.head()

,game_id,winner,duration,train_sum_0,build_sum_0,upgrade_sum_0,attack_sum_0,supply_0,minerals_0,gas_0,map_0,map_1,map_2,map_3,map_4,map_5,map_6,train_sum_1,build_sum_1,upgrade_sum_1,attack_sum_1,supply_1,minerals_1,gas_1,species0_T,species0_P,species0_Z,species1_T,species1_P,species1_Z,player0_near0,player0_near1,player1_near1,player1_near0,player0_click_near0,player0_click_near1,player1_click_near1,player1_click_near0,cam_remote_ratio0,cam_remote_ratio1,cam_remote_ratio0_last,cam_remote_ratio1_last,camera_home_p1,camera_home_p0,camera_enemy_to_p1,camera_enemy_to_p0,homeaway_p0,homeaway_p1,p0_moving_mean,p1_moving_mean,p0_moving_std,p1_moving_std,p0_attack_units,p1_attack_units,dist_between_0_to_mbf_total,dist_between_1_to_mbf_total,p0_actions,p1_actions
0,0,1.00,7.22,13.00,15.00,2.00,0.00,16.00,775.00,125.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,11.00,11.00,2.00,3.00,17.00,850.00,250.00,1.00,0.00,0.00,1.00,0.00,0.00,406.00,37.00,423.00,0.00,22.00,13.00,28.00,0.00,0.11,0.00,0.00,0.00,428.00,453.00,51.00,0.00,0.00,0.12,4.81,2.38,15.35,2.02,0.00,12.00,119.75,100.68,69.00,62.00
1,1,1.00,9.54,37.00,23.00,4.00,2.00,42.00,2200.00,625.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,25.00,21.00,3.00,4.00,37.00,1825.00,550.00,0.00,1.00,0.00,1.00,0.00,0.00,545.00,14.00,629.00,92.00,110.00,26.00,76.00,27.00,0.11,0.25,0.46,0.08,699.00,746.00,88.00,243.00,0.33,0.13,7.39,8.52,17.38,26.45,4.00,18.00,91.00,123.91,237.00,198.00
2,2,0.00,9.54,26.00,21.00,3.00,11.00,32.00,1675.00,475.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,39.00,15.00,4.00,7.00,35.00,2600.00,100.00,0.00,1.00,0.00,0.00,0.00,1.00,259.00,26.00,529.00,62.00,82.00,22.00,123.00,32.00,0.18,0.18,0.35,0.06,467.00,758.00,106.00,171.00,0.23,0.23,13.25,8.46,23.32,20.06,36.00,10.50,128.58,124.20,229.00,289.00
3,3,0.00,9.48,25.00,20.00,4.00,8.00,39.00,1925.00,750.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,37.00,20.00,2.00,17.00,43.00,2400.00,525.00,1.00,0.00,0.00,0.00,1.00,0.00,539.00,127.00,250.00,67.00,169.00,77.00,177.00,34.00,0.26,0.30,0.06,0.92,733.00,456.00,256.00,200.00,0.44,0.35,8.28,10.69,20.28,22.57,24.00,37.00,104.69,79.06,358.00,357.00
4,4,0.00,5.35,18.00,10.00,2.00,14.00,22.00,1100.00,100.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,20.00,5.00,1.00,2.00,17.50,1375.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,340.00,67.00,207.00,1.00,64.00,85.00,69.00,11.00,0.25,0.16,0.00,0.07,455.00,318.00,152.00,60.00,0.19,0.33,6.10,20.31,16.61,27.16,13.00,0.00,139.39,158.01,234.00,142.00


In [39]:
df = pd.read_csv('submission_baseline_0414_4_inner_inner.csv')

In [40]:
df.shape

(16787, 2)

In [41]:
df.head()

,game_id,winner
0,38872,0.68
1,38873,0.50
2,38874,0.30
3,38875,0.21
4,38876,0.50


In [43]:
df[df.winner == 0.5]

,game_id,winner
1,38873,0.50
4,38876,0.50
5,38877,0.50
6,38878,0.50
31,38903,0.50
...,...,...
16751,55623,0.50
16766,55638,0.50
16768,55640,0.50
16784,55656,0.50


In [49]:
len(list(pd.read_csv('data/unit_select_raw_train.csv').columns))

111

In [50]:
len(list(pd.read_csv('data/unit_select_raw_test.csv').columns))

111

In [22]:
df_2 = pd.read_csv('model_data/train_final_ftr_0414_3.csv')

In [23]:
df_2.shape

(38872, 159)

In [24]:
df_2.head()

,game_id,Adept,Archon,Baneling,Banshee,Battlecruiser,BroodLord,Carrier,Colossus,Corruptor,Cyclone,DarkTemplar,Disruptor,Drone,Ghost,Hellion,HighTemplar,Hydralisk,Immortal,Infestor,Interceptor,Liberator,Lurker,Marauder,Marine,Medivac,Mothership,MothershipCore,Mutalisk,Nuke,Observer,Oracle,Overlord,Overseer,Phoenix,Probe,Queen,Ravager,Raven,Reaper,Roach,SCV,Sentry,SiegeTank,Stalker,SwarmHost,Tempest,Thor,Ultralisk,Viking,Viper,VoidRay,WarpPrism,WidowMine,Zealot,Zergling,Adept.1,Archon.1,Baneling.1,Banshee.1,Battlecruiser.1,BroodLord.1,Carrier.1,Colossus.1,Corruptor.1,Cyclone.1,DarkTemplar.1,Disruptor.1,Drone.1,Ghost.1,Hellion.1,HighTemplar.1,Hydralisk.1,Immortal.1,Infestor.1,Interceptor.1,Liberator.1,Lurker.1,Marauder.1,Marine.1,Medivac.1,Mothership.1,MothershipCore.1,Mutalisk.1,Nuke.1,Observer.1,Oracle.1,Overlord.1,Overseer.1,Phoenix.1,Probe.1,Queen.1,Ravager.1,Raven.1,Reaper.1,Roach.1,SCV.1,Sentry.1,SiegeTank.1,Stalker.1,SwarmHost.1,Tempest.1,Thor.1,Ultralisk.1,Viking.1,Viper.1,VoidRay.1,WarpPrism.1,WidowMine.1,Zealot.1,Zergling.1,winner,duration,species_0,train_sum_0,build_sum_0,upgrade_sum_0,attack_sum_0,supply_0,minerals_0,gas_0,map_0,species_1,train_sum_1,build_sum_1,upgrade_sum_1,attack_sum_1,supply_1,minerals_1,gas_1,map_1,player0_near0,player0_near1,player1_near1,player1_near0,player0_click_near0,player0_click_near1,player1_click_near1,player1_click_near0,cam_remote_ratio0,cam_remote_ratio1,cam_remote_ratio0_last,cam_remote_ratio1_last,camera_home_p1,camera_home_p0,camera_enemy_to_p1,camera_enemy_to_p0,homeaway_p0,homeaway_p1,p0_moving_mean,p1_moving_mean,p0_moving_std,p1_moving_std,p0_attack_units,p1_attack_units,dist_between_0_to_mbf_total,dist_between_1_to_mbf_total,p0_actions,p1_actions
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.00,7.22,1,13.00,15.00,2.00,0.00,16.00,775.00,125.00,1,1,11.00,11.00,2.00,3.00,17.00,850.00,250.00,1,406.00,37.00,423.00,0.00,22.00,13.00,28.00,0.00,0.11,0.00,0.00,0.00,428.00,453.00,51.00,0.00,0.00,0.12,4.81,2.38,15.35,2.02,0.00,12.00,119.75,100.68,69.00,62.00
1,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.00,9.54,2,37.00,23.00,4.00,2.00,42.00,2200.00,625.00,4,1,25.00,21.00,3.00,4.00,37.00,1825.00,550.00,4,545.00,14.00,629.00,92.00,110.00,26.00,76.00,27.00,0.11,0.25,0.46,0.08,699.00,746.00,88.00,243.00,0.33,0.13,7.39,8.52,17.38,26.45,4.00,18.00,91.00,123.91,237.00,198.00
2,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,0.00,9.54,2,26.00,21.00,3.00,11.00,32.00,1675.00,475.00,6,3,39.00,15.00,4.00,7.00,35.00,2600.00,100.00,6,259.00,26.00,529.00,62.00,82.00,22.00,123.00,32.00,0.18,0.18,0.35,0.06,467.00,758.00,106.00,171.00,0.23,0.23,13.25,8.46,23.32,20.06,36.00,10.50,128.58,124.20,229.00,289.00
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,18,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,1,0,0,0,0.00,9.48,1,25.00,20.00,4.00,8.00,39.00,1925.00,750.00,2,2,37.00,20.00,2.00,17.00,43.00,2400.00,525.00,2,539.00,127.00,250.00,67.00,169.00,77.00,177.00,34.00,0.26,0.30,0.06,0.92,733.00,456.00,256.00,200.00,0.44,0.35,8.28,10.69,20.28,22.57,24.00,37.00,104.69,79.06,358.00,357.00
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.00,5.35,1,18.00,10.00,2.00,14.00,22.00,1100.00,100.00,5,3

In [79]:
df_2.map_0.unique()

array([1, 4, 6, 2, 5, 3, 0])

In [302]:
train_final_ftr_df.shape

(38676, 256)

In [273]:
train_final_ftr_df['species_mix'] = train_final_ftr_df.apply(lambda x: str(x.species_0) + str(x.species_1), axis='columns')

In [274]:
train_final_ftr_df.shape

(38676, 235)

In [275]:
train_final_ftr_df.species_mix.value_counts()

11    5636
31    5173
13    5094
21    4363
12    4290
33    3992
32    3663
23    3530
22    2935
Name: species_mix, dtype: int64

In [276]:
train_final_ftr_df_11 = train_final_ftr_df[train_final_ftr_df['species_mix'] == '11']

In [277]:
train_final_ftr_df = train_final_ftr_df_11

In [278]:
train_final_ftr_df.shape

(5636, 235)

In [13]:
train_data, test_data = train_test_split(train_final_ftr_df, test_size=0.2, random_state=1)
train_data, valid_data = train_test_split(train_data, test_size=0.2, random_state=1)

X_train, y_train = train_data[_ftr], train_data['winner']
X_valid, y_valid = valid_data[_ftr], valid_data['winner']
X_test, y_test = test_data[_ftr], test_data['winner']

lgb_train_data = lgb.Dataset(X_train, label=y_train)
lgb_valid_data = lgb.Dataset(X_valid, label=y_valid)
lgb_test_data = lgb.Dataset(X_test, label=y_test)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(24877, 157)
(6220, 157)
(7775, 157)


In [55]:
train_data, test_data = train_test_split(train_final_ftr_df, test_size=0.2, random_state=1)

X_train, y_train = train_data[_ftr], train_data['winner']
X_test, y_test = test_data[_ftr], test_data['winner']

lgb_train_data = lgb.Dataset(X_train, label=y_train)
lgb_test_data = lgb.Dataset(X_test, label=y_test)

print(X_train.shape)
print(X_test.shape)

(31097, 56)
(7775, 56)


In [56]:
# 7859
# params = {
#     'boosting_type': 'dart',
#     'n_jobs': 4,
# #     'objective': 'binary',
# #     'metrics': 'auc',
    
#     'num_leaves': 26,        # num_leaves,       범위(16~1024)
#     'learning_rate': 0.004814,  # learning_rate,    범위(0.0001~0.1)
#     'num_iterations': 7859,      # n_estimators,     범위(16~1024)
#     'bagging_fraction': 0.6347,             # subsample,        범위(0~1)
#     'feature_fraction': 0.4961,      # colsample_bytree, 범위(0~1)
#     'lambda_l1': 6.454,            # reg_alpha,        범위(0~10)
#     'lambda_l2': 0.108,           # reg_lambda,       범위(0~50)
# }

# 0.6833
# params = {
#     'boosting_type': 'gbdt',
#     'n_jobs': 4,
#     'objective': 'binary',
#     'metrics': 'auc',
    
#     'num_leaves': 19,        # num_leaves,       범위(16~1024)
#     'learning_rate': 0.02794,  # learning_rate,    범위(0.0001~0.1)
#     'n_estimators': 724,      # n_estimators,     범위(16~1024)
#     'subsample': 0.8579,             # subsample,        범위(0~1)
#     'colsample_bytree': 0.8448,      # colsample_bytree, 범위(0~1)
#     'reg_alpha': 2.816,            # reg_alpha,        범위(0~10)
#     'reg_lambda': 1.524,           # reg_lambda,       범위(0~50)
# }

# # 0410
# params = {
#     'boosting_type': 'gbdt',
#     'n_jobs': 4,
#     'objective': 'binary',
#     'metrics': 'auc',
#     'weight_column': 'p0_attack_units',

#     'bagging_fraction': 1.0,
#     'feature_fraction': 0.713359337126435,
#     'lambda_l1': 9.225099197025434,
#     'lambda_l2': 49.43242484634145,
#     'learning_rate': 0.06242146704630103,
#     'num_iterations': 774,
#     'num_leaves': 91
# }

# # 0410
# params = {
#     'boosting_type': 'gbdt',
#     'n_jobs': 4,
#     'objective': 'binary',
#     'metrics': 'auc',

#     'num_iterations': 774,
# }

# # 0414_2
# params = {'boosting_type': 'dart',
#           'n_jobs': 4,
#           'objective': 'binary',
#           'metrics': 'auc',
          
#           'bagging_fraction': 1.0, 
#           'feature_fraction': 0.5218, 
#           'lambda_l1': 7.437, 
#           'lambda_l2': 46.2, 
#           'learning_rate': 0.08819, 
#           'min_data_in_leaf': 33, 
#           'num_iterations': 998, 
#           'num_leaves': 1013}

# # 0414_3
# params = {'boosting_type': 'dart',
#           'n_jobs': 4,
#           'objective': 'binary',
#           'metrics': 'auc',
          
#           'bagging_fraction': 1.0, 
#           'feature_fraction': 0.5868183544272747, 
#           'lambda_l1': 2.27974900764249, 
#           'lambda_l2': 47.847072146440915, 
#           'learning_rate': 0.053310375663659613, 
#           'min_data_in_leaf': 30, 
#           'num_iterations': 934, 
#           'num_leaves': 254}

# 0414_4
params = {'boosting_type': 'dart',
          'n_jobs': 4,
          'objective': 'binary',
          'metrics': 'auc',
          
#           'categorical_feature': ['map_0','species_0','species_1'],
          
#             'bagging_fraction': 0.9339555404817057,
            'bagging_fraction': 1,
            'feature_fraction': 0.9852242346443292, 
            'lambda_l1': 2.9659344859213888, 
            'lambda_l2': 46.14165176383672, 
            'learning_rate': 0.06458834869249273, 
            'min_data_in_leaf': 13, 
            'num_iterations': 990, 
            'num_leaves': 20
         }

bst = lgb.train(params, lgb_train_data,
#                valid_sets=[lgb_valid_data],
#                early_stopping_rounds=50,
#                categorical_feature=['map_0','species_0','species_1'],
               verbose_eval=100
               )

In [57]:
predict_test = bst.predict(X_train)

In [58]:
predict_test

array([0.26183872, 0.56554769, 0.43376256, ..., 0.29884149, 0.66226759,
       0.41059407])

In [29]:
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.9292212876925636


In [169]:
# 0410
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.950155778800241


In [195]:
# 7859
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.8511803437248723


In [216]:
# 0.6833
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.8643068162669667


In [283]:
# 0.6833
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.9901384737510565


In [313]:
# 0.6833
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.9352702538523712


In [9]:
# 0414_2
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.9968882288422494


In [9]:
# 0414_3
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.9858736460821573


In [17]:
# 0414_3 gbdt early
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.9723466722441879


In [32]:
# 0414_4
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.795601412278831


In [61]:
# 0414_4
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.7954472059113324


In [8]:
# 0414_5
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.7933222727768348


In [25]:
# 0414_5_i
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.8106858112902444


In [59]:
# 0414_5
auc = roc_auc_score(y_train, predict_test)
print(auc)

0.7878789162336782


In [60]:
feature_importance = pd.DataFrame([bst.feature_name(), bst.feature_importance()]).T
feature_importance.columns = ['ftr', 'ftr_imp']
feature_importance = feature_importance.sort_values(['ftr_imp'], ascending=False)

feature_importance

,ftr,ftr_imp
20,minerals_1,954
6,minerals_0,949
2,build_sum_0,903
46,p0_moving_mean,867
16,build_sum_1,833
47,p1_moving_mean,805
40,camera_home_p1,684
0,duration,628
15,train_sum_1,596
1,train_sum_0,523


In [61]:
predict_test = bst.predict(X_test)

In [62]:
predict_test

array([0.27726869, 0.51306833, 0.60801635, ..., 0.38175669, 0.48704196,
       0.53604849])

In [35]:
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6723379561862306


In [172]:
# 0410
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6697081755598636


In [198]:
# 7859
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6697437339101806


In [219]:
# 0.6833
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6674915268194179


In [287]:
# 0.6833
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6592908247503734


In [316]:
# 0.6833
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6741252311626965


In [12]:
# 0414_2
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6694620516689036


In [12]:
# 0414_3
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6686643316481766


In [20]:
# 0414_3 gbdt early
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6717314264486387


In [65]:
# 0414_4
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6699079434670628


In [12]:
# 0414_5
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6694281596557419


In [29]:
# 0414_5_i
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6692954119715561


In [36]:
# 0414_5_i
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.754885844815356


In [63]:
# 0414_5
auc = roc_auc_score(y_test, predict_test)
print(auc)

0.6589912730713923


In [220]:
y_test

21121    0
31017    0
1266     1
25861    1
22506    0
        ..
23310    0
24969    0
26927    0
31808    1
10429    0
Name: winner, Length: 7736, dtype: int64

In [221]:
predict_test

array([0.33168944, 0.27182029, 0.34899105, ..., 0.36720616, 0.58105226,
       0.27699021])

In [230]:
# 예측이 틀린 게임 번호
wrong_game_id_list = y_test[abs(y_test - predict_test) > 0.5].index.tolist()

In [231]:
len(wrong_game_id_list)

2961

In [242]:
sorted(wrong_game_id_list)

[3,
 11,
 33,
 40,
 44,
 45,
 65,
 99,
 105,
 121,
 134,
 142,
 149,
 151,
 175,
 192,
 209,
 221,
 230,
 271,
 275,
 285,
 286,
 292,
 294,
 300,
 321,
 364,
 370,
 377,
 383,
 395,
 421,
 452,
 456,
 483,
 498,
 501,
 512,
 537,
 542,
 565,
 567,
 627,
 633,
 636,
 653,
 654,
 685,
 705,
 714,
 738,
 743,
 757,
 766,
 788,
 789,
 815,
 825,
 832,
 848,
 852,
 859,
 860,
 868,
 869,
 871,
 874,
 881,
 896,
 902,
 911,
 917,
 948,
 950,
 953,
 969,
 992,
 1024,
 1026,
 1059,
 1077,
 1086,
 1094,
 1095,
 1117,
 1132,
 1133,
 1141,
 1158,
 1169,
 1189,
 1214,
 1216,
 1228,
 1239,
 1246,
 1266,
 1281,
 1289,
 1294,
 1303,
 1305,
 1313,
 1320,
 1338,
 1362,
 1366,
 1383,
 1401,
 1425,
 1428,
 1436,
 1463,
 1472,
 1477,
 1482,
 1493,
 1494,
 1498,
 1519,
 1520,
 1521,
 1544,
 1593,
 1621,
 1633,
 1637,
 1657,
 1676,
 1686,
 1703,
 1709,
 1726,
 1746,
 1758,
 1766,
 1773,
 1781,
 1783,
 1807,
 1814,
 1841,
 1852,
 1854,
 1863,
 1886,
 1898,
 1899,
 1916,
 1918,
 1929,
 1933,
 1942,
 1966,
 1

In [236]:
train_final_ftr_df.iloc[[0]]

,Unnamed__0,game_id,winner,duration,species_0,train_sum_0,build_sum_0,upgrade_sum_0,attack_sum_0,supply_0,minerals_0,gas_0,map_0,species_1,train_sum_1,build_sum_1,upgrade_sum_1,attack_sum_1,supply_1,minerals_1,gas_1,map_1,Adept,Archon,Baneling,Banshee,Battlecruiser,BroodLord,Carrier,Colossus,Corruptor,Cyclone,DarkTemplar,Disruptor,Drone,Ghost,Hellion,HighTemplar,Hydralisk,Immortal,Infestor,Interceptor,Liberator,Lurker,Marauder,Marine,Medivac,Mothership,MothershipCore,Mutalisk,Nuke,Observer,Oracle,Overlord,Overseer,Phoenix,Probe,Queen,Ravager,Raven,Reaper,Roach,SCV,Sentry,SiegeTank,Stalker,SwarmHost,Tempest,Thor,Ultralisk,Viking,Viper,VoidRay,WarpPrism,WidowMine,Zealot,Zergling,Adept_1,Archon_1,Baneling_1,Banshee_1,Battlecruiser_1,BroodLord_1,Carrier_1,Colossus_1,Corruptor_1,Cyclone_1,DarkTemplar_1,Disruptor_1,Drone_1,Ghost_1,Hellion_1,HighTemplar_1,Hydralisk_1,Immortal_1,Infestor_1,Interceptor_1,Liberator_1,Lurker_1,Marauder_1,Marine_1,Medivac_1,Mothership_1,MothershipCore_1,Mutalisk_1,Nuke_1,Observer_1,Oracle_1,Overlord_1,Overseer_1,Phoenix_1,Probe_1,Queen_1,Ravager_1,Raven_1,Reaper_1,Roach_1,SCV_1,Sentry_1,SiegeTank_1,Stalker_1,SwarmHost_1,Tempest_1,Thor_1,Ultralisk_1,Viking_1,Viper_1,VoidRay_1,WarpPrism_1,WidowMine_1,Zealot_1,Zergling_1,player0_near0,player0_near1,player1_near1,player1_near0,player0_click_near0,player0_click_near1,player1_click_near1,player1_click_near0,cam_remote_ratio0,cam_remote_ratio1,cam_remote_ratio0_last,cam_remote_ratio1_last,camera_home_p1,camera_home_p0,camera_enemy_to_p1,camera_enemy_to_p0,homeaway_p0,homeaway_p1,p0_center_0,p0_center_1,p0_center_2,p0_center_3,p0_center_4,p0_center_5,p0_center_6,p0_center_7,p0_center_8,p0_center_9,p1_center_0,p1_center_1,p1_center_2,p1_center_3,p1_center_4,p1_center_5,p1_center_6,p1_center_7,p1_center_8,p1_center_9,p0_actions_0,p0_actions_1,p0_actions_2,p0_actions_3,p0_actions_4,p0_actions_5,p0_actions_6,p0_actions_7,p0_actions_8,p0_actions_9,p1_actions_0,p1_actions_1,p1_actions_2,p1_actions_3,p1_actions_4,p1_actions_5,p1_actions_6,p1_actions_7,p1_actions_8,p1_actions_9,p0_moving_mean,p1_moving_mean,p0_moving_std,p1_moving_std,cam_moving_p0_mean0,cam_moving_p0_mean1,cam_moving_p0_mean2,cam_moving_p0_mean3,cam_moving_p0_mean4,cam_moving_p0_mean5,cam_moving_p0_mean6,cam_moving_p0_mean7,cam_moving_p0_mean8,cam_moving_p0_mean9,cam_moving_p1_mean0,cam_moving_p1_mean1,cam_moving_p1_mean2,cam_moving_p1_mean3,cam_moving_p1_mean4,cam_moving_p1_mean5,cam_moving_p1_mean6,cam_moving_p1_mean7,cam_moving_p1_mean8,cam_moving_p1_mean9,cam_moving_p0_std0,cam_moving_p0_std1,cam_moving_p0_std2,cam_moving_p0_std3,cam_moving_p0_std4,cam_moving_p0_std5,cam_moving_p0_std6,cam_moving_p0_std7,cam_moving_p0_std8,cam_moving_p0_std9,cam_moving_p1_std0,cam_moving_p1_std1,cam_moving_p1_std2,cam_moving_p1_std3,cam_moving_p1_std4,cam_moving_p1_std5,cam_moving_p1_std6,cam_moving_p1_std7,cam_moving_p1_std8,cam_moving_p1_std9
0,0,0,1,7.22,1,13.00,15.00,2.00,0.00,16.00,775.00,125.00,1,1,11.00,11.00,2.00,3.00,17.00,850.00,250.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,406.00,37.00,423.00,0.00,22.00,13.00,28.00,0.00,0.11,0.00,0.00,0.00,428.00,453.00,51.00,0.00,0.00,0.12,13.53,29.49,15.56,28.03,94.50,15.95,24.79,19.32,0.00,0.00,4.58,2.29,11.10,14.58,13.48,14.03,14.68,13.74,0.00,0.00,6.00,11.00,7.00,8.00,13.00,7.00,13.00,4.00,0.00,0.00,4.00,2.00,5.00,8.00,15.00,14.00,9.00,5.00,0.00,0.00,4.81,2.38,15.35,2.02,2.59,16.13,2.92,4.08,5.49,2.84,3.46,3.90,0.00,0.00,1.79,2.02,1.91,2.14,2.57,2.96,2.67,2.42,0.00,0.00,1.49,41.11,3.17,13.60,15.37,2.45,3.55,4.05,0.00,0.00,1.06,2.67,1.09,1.61,2.19,2.24,2.51,2.50,0.00,0.00


In [243]:
wrong_game_df = train_final_ftr_df[train_final_ftr_df.game_id.isin(wrong_game_id_list)]

In [264]:
wrong_game_df.apply(lambda x: str(x.species_0) + str(x.species_1), axis='columns').value_counts()

11    415
13    408
31    401
21    351
12    309
33    288
23    281
22    249
32    247
dtype: int64

In [251]:
train_final_ftr_df.map_1.value_counts()

6    6582
1    6543
3    5689
2    5546
5    5297
4    5145
0    3874
Name: map_1, dtype: int64

In [265]:
train_final_ftr_df.apply(lambda x: str(x.species_0) + str(x.species_1), axis='columns').value_counts()

11    5636
31    5173
13    5094
21    4363
12    4290
33    3992
32    3663
23    3530
22    2935
dtype: int64

In [13]:
# bst.save_model(os.path.join(data_folder,'baseline_0303.model'))



#### > test_df에서 누락된 game_id 추가 (null인 game_id의 winner는 0.5로)

_ftr = test_final_ftr_df.columns.tolist()
_ftr.remove('game_id')

predict_test = bst.predict(test_final_ftr_df[_ftr])


# print('[n_of_game_id]')
# print('train_df: ', len(train_df.game_id.unique()))
# print('train_ability_feature_df: ', len(train_final_ftr_df.game_id.unique()))
# print('test_df: ', len(test_df.game_id.unique()))
# print('predict_test: ', len(predict_test))

predict_test_df = pd.DataFrame(predict_test).reset_index().rename(columns={0: 'winner'})

test_game_id_df = test_final_ftr_df[['game_id']].reset_index()

predict_test_small_df = test_game_id_df.merge(predict_test_df, how='inner', on='index').drop('index', axis='columns')

# print('[n_of_game_id]')
# print('train_df: ', len(train_df.game_id.unique()))
# print('train_ability_feature_df: ', len(train_final_ftr_df.game_id.unique()))
# print('test_df: ', len(test_df.game_id.unique()))
# print('predict_test_small_df: ', len(predict_test_small_df))

###
test_df = pd.read_csv('data/sample_submission.csv')
###

original_test_game_id_df = pd.DataFrame(test_df.game_id.unique(), columns=['game_id'])

result_df = original_test_game_id_df.merge(predict_test_small_df, how='left', on='game_id').fillna(0.5)


result_df.head()

plt.hist(result_df.winner)

 # Output

# result_df.to_csv('submission_baseline_0413_2.csv', index=False)

NameError: name 'test_final_ftr_df' is not defined